In [1]:
cd Desktop/sp19-222-92/project/gamedata

/Users/benyeagley/Desktop/sp19-222-92/project/gamedata


In [2]:
import pandas as pd
import numpy as np

In [3]:
data = pd.read_csv("total_game_data2.csv")
data.head()

,HR min [bpm],HR avg [bpm],HR max [bpm],HR min [%],HR avg [%],HR max [%],Time in HR zone 1 (50 - 59 %),Time in HR zone 2 (60 - 69 %),Time in HR zone 3 (70 - 79 %),Time in HR zone 4 (80 - 89 %),...,Number of accelerations (-1.99 - -1.00 m/s),Number of accelerations (-0.99 - -0.50 m/s),Number of accelerations (0.50 - 0.99 m/s),Number of accelerations (1.00 - 1.99 m/s),Number of accelerations (2.00 - 2.99 m/s),Number of accelerations (3.00 - 50.00 m/s),Calories [kcal],Training load score,Recovery time [h],Min Played
0,37,148,210,19,74,105,0:19:22,0:27:11,0:56:50,0:52:40,...,399,762,593,389,119,19,1343,230,46.0,99
1,62,98,188,31,49,94,0:23:49,0:08:11,0:08:23,0:02:33,...,82,239,211,69,12,0,495,30,5.5,0
2,80,150,217,40,75,109,0:28:32,0:20:40,0:15:03,0:35:51,...,302,601,500,333,97,11,1407,289,135.7,82
3,86,150,200,43,75,100,0:19:42,0:15:29,0:19:19,0:35:16,...,352,590,647,360,77,12,1462,302,151.4,79
4,83,121,192,42,61,96,1:23:38,0:25:26,0:11:26,0:10:58,...,129,325,276,118,21,7,1019,118,21.3,7


In [4]:
# Removes every row where a player had zero minutes played
data = data[data['Min Played'] != 0]

In [5]:
data = data[data.columns.drop(list(data.filter(regex='Time')))]
data = data[data.columns.drop(list(data.filter(regex='HR')))]
data = data[data.columns.drop(list(data.filter(regex='Calories')))]
data = data[data.columns.drop(list(data.filter(regex='load score')))]
data = data[data.columns.drop(list(data.filter(regex='Recovery')))]
data = data[data.columns.drop(list(data.filter(regex='Speed zone 1')))]

data.head()

,Total distance [yd],Distance / min [yd/min],Maximum speed [mph],Average speed [mph],Sprints,Distance in Speed zone 2 [yd] (0.10 - 2.59 mph),Distance in Speed zone 3 [yd] (2.60 - 5.13 mph),Distance in Speed zone 4 [yd] (5.14 - 8.38 mph),Distance in Speed zone 5 [yd] (8.39- mph),Number of accelerations (-50.00 - -3.00 m/s),Number of accelerations (-2.99 - -2.00 m/s),Number of accelerations (-1.99 - -1.00 m/s),Number of accelerations (-0.99 - -0.50 m/s),Number of accelerations (0.50 - 0.99 m/s),Number of accelerations (1.00 - 1.99 m/s),Number of accelerations (2.00 - 2.99 m/s),Number of accelerations (3.00 - 50.00 m/s),Min Played
0,15935,98,18.7,3.5,37,1595,4651,5321,4366,27,72,399,762,593,389,119,19,99
2,11568,71,18.3,2.6,20,1464,3483,3802,2815,27,109,302,601,500,333,97,11,82
3,11981,74,16.4,2.7,23,1503,3750,3758,2969,25,94,352,590,647,360,77,12,79
4,3133,19,16.3,0.7,9,683,783,1034,628,7,25,129,325,276,118,21,7,7
5,13265,81,17.9,2.9,19,1501,5030,4573,2158,28,84,332,671,629,379,78,14,92


In [6]:
#data['Sprints'].apply(np.float64)
data['Sprints'] = data['Sprints'].apply(np.float64)
data.info()
# Adjust sprints for minutes played
for index, row in data.iterrows():
    print(index)
    # Change # of sprints to sprints per min
    data.at[index, 'Sprints'] = row['Sprints'] / row['Min Played']
    for i in range(5,9):
        data.at[index, i] = (row[i]) / int(row[0])
        
data.head()
data.to_csv("altered_total.csv")

<class 'pandas.core.frame.DataFrame'>
Int64Index: 170 entries, 0 to 231
Data columns (total 18 columns):
Total distance [yd]                                170 non-null int64
Distance / min [yd/min]                            170 non-null int64
Maximum speed [mph]                                170 non-null float64
Average speed [mph]                                170 non-null float64
Sprints                                            170 non-null float64
Distance in Speed zone 2 [yd] (0.10 - 2.59 mph)    170 non-null int64
Distance in Speed zone 3 [yd] (2.60 - 5.13 mph)    170 non-null int64
Distance in Speed zone 4 [yd] (5.14 - 8.38 mph)    170 non-null int64
Distance in Speed zone 5 [yd] (8.39- mph)          170 non-null int64
Number of accelerations (-50.00 - -3.00 m/s)       170 non-null int64
Number of accelerations (-2.99 - -2.00 m/s)        170 non-null int64
Number of accelerations (-1.99 - -1.00 m/s)        170 non-null int64
Number of accelerations (-0.99 - -0.50 m/s)      

In [7]:
# Normalize
from sklearn import preprocessing

x = data.values #returns a numpy array
min_max_scaler = preprocessing.MinMaxScaler()
x_scaled = min_max_scaler.fit_transform(x)
data = pd.DataFrame(x_scaled)

In [8]:
data.head()

,0,1,2,3,4,5,6,7,8,9,...,688,689,690,691,692,693,694,695,696,697
0,0.968666,1.000000,0.177370,0.93750,0.146825,0.411714,0.836563,0.825138,1.000000,0.702703,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.651918,0.682353,0.165138,0.65625,0.095819,0.355297,0.585056,0.555477,0.623452,0.702703,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.681874,0.717647,0.107034,0.68750,0.114376,0.372093,0.642550,0.547666,0.660840,0.648649,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.040110,0.070588,0.103976,0.06250,0.505102,0.018949,0.003661,0.064087,0.092498,0.162162,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.775005,0.800000,0.152905,0.75000,0.081134,0.371232,0.918174,0.692349,0.463948,0.729730,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
